# Simon Fraser University - Mechatronic Systems Engineering
## Spring 2021 - MSE491 - Application of Machine Learning in Mechatronic Systems
## Instructor: Dr. Mohammad Narimani

## Lab 2 - Classification
### Presenter: Samira Asadi - saa143@sfu.ca


In [ ]:
# Load General Libraries
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

# Importing the dataset (no header)
df0 = pd.read_csv("EMG\EMG_0.csv", header=None)
df1 = pd.read_csv("EMG\EMG_1.csv", header=None)
df2 = pd.read_csv("EMG\EMG_2.csv", header=None)
df3 = pd.read_csv("EMG\EMG_3.csv", header=None)
dataset = pd.concat([df0,df1,df2,df3], axis = 0)

# Importing hand gestures images
import matplotlib.image as mpimg
img0 = mpimg.imread('EMG/0.jpg')
img1 = mpimg.imread('EMG/1.jpg')
img2 = mpimg.imread('EMG/2.jpg')
img3 = mpimg.imread('EMG/3.jpg')

# Binary Classification

In [ ]:
# Creating a Binary Dataset (by only considering 2 labels)
# Class 0: Closed Fist
# Class 1: Open Hand

dataset_binary = dataset[dataset.iloc[:, -1].isin([0,2])]
dataset_binary.iloc[:, -1].replace(to_replace=2, value=1, inplace=True)

# Split features and targets - X: Features, y: Targets
X_binary = dataset_binary.iloc[:, :-1]
y_binary = dataset_binary.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(X_binary, y_binary, test_size=0.2)

# Save Test Set
np.savetxt("features_emg_test_binary.csv", X_test_binary, delimiter=",")
np.savetxt("targets_emg_test_binary.csv", y_test_binary, delimiter=",")

# Standardize features by removing the mean and scaling to unit variance
from sklearn.preprocessing import StandardScaler
X_train_binary = pd.DataFrame(StandardScaler().fit_transform(X_train_binary))
X_test_binary = pd.DataFrame(StandardScaler().fit_transform(X_test_binary))

In [ ]:
# Display what each label refers to - Binary Classification
fig = plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(img0)
plt.axis('off')
plt.title('0 - Closed Fist')
plt.subplot(1, 2, 2)
plt.imshow(img2)
plt.axis('off')
plt.title('1 - Open Hand')
plt.show()

## Test the difference between the mean values of two features from different classes

In [ ]:
from scipy.stats import ttest_ind

FEATURES_C1 = dataset_binary[dataset_binary.iloc[:, -1]==0] # features for closed fist
FEATURES_C2 = dataset_binary[dataset_binary.iloc[:, -1]==1] # features for open hand

# Test the significance
T_STATISTIC, P_VALUE = ttest_ind(FEATURES_C1.iloc[:, 0],FEATURES_C2.iloc[:, 0])

print("p-value =", P_VALUE)
# if the p-value is less than 0.05 we have significant different between feature values
if P_VALUE < 0.05:
    print("significant")
else:
    print("not significant")

## Logistic Regression

In [ ]:
# Import the class
from sklearn.linear_model import LogisticRegression

# Instantiating the model (using the default parameters)
logreg = LogisticRegression()

# Fitting the model with data
logreg.fit(X_train_binary,y_train_binary)

# Save the Trained Model
pickle.dump(logreg, open('Model_LogisticRegression.pkl', 'wb'))

# Predicting the labels on test set
y_pred_binary = logreg.predict(X_test_binary)

## Model Evaluation using Confusion Matrix

In [ ]:
# import the metrics class
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
cnf_matrix = confusion_matrix(y_test_binary, y_pred_binary)
print('Confusion Matrix\n', cnf_matrix)

plt_cnf_matrix = plot_confusion_matrix(logreg, X_test_binary, y_test_binary,
                                       display_labels=['Closed Fist','Open Hand'],
                                       cmap=plt.cm.Blues,
                                       normalize='true')
plt_cnf_matrix.ax_.set_title('Confusion Matrix - Binary Classification')

# Print Evaluation Metrics
print("Accuracy:",accuracy_score(y_test_binary, y_pred_binary))
print("Precision:",precision_score(y_test_binary, y_pred_binary))
print("Recall:",recall_score(y_test_binary, y_pred_binary))
print("F1 Score:",f1_score(y_test_binary, y_pred_binary))

# Multi-Class Classification

In [ ]:
# Multi-class dataset
# Class 0: Closed Fist
# Class 1: V Sign
# Class 2: Open Hand
# Class 3: OK Gesture
# Split features and targets - X: Features, y: Targets
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Save Test Set
np.savetxt("features_emg_test.csv", X_test, delimiter=",")
np.savetxt("targets_emg_test.csv", y_test, delimiter=",")

# Standardize features by removing the mean and scaling to unit variance
from sklearn.preprocessing import StandardScaler
X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))

In [ ]:
print(X_train.shape)

In [ ]:
# Display what each label refers to - Multi-class Classification
fig = plt.figure()
plt.subplot(2, 2, 1)
plt.imshow(img0)
plt.axis('off')
plt.title('0 - Closed Fist')
plt.subplot(2, 2, 2)
plt.imshow(img1)
plt.axis('off')
plt.title('1 - V Sign')
plt.subplot(2, 2, 3)
plt.imshow(img2)
plt.axis('off')
plt.title('2 - Open Hand')
plt.subplot(2, 2, 4)
plt.imshow(img3)
plt.axis('off')
plt.title('3 - OK Gesture')
plt.show()

# Classification Models

In [ ]:
# Some Functions for Showing the Classifier Performance
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

labels = ['Closed Fist','V Sign','Open Hand','OK Gesture']
def classifier_performance(model,y_pred):
    print('Classification Report: \n', classification_report(y_test,y_pred,target_names=labels))
    # Plot normalized confusion matrix
    titles_options = [("Confusion matrix, without normalization", None),
                      ("Normalized confusion matrix", 'true')]
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(model, X_test, y_test,
                                     display_labels=labels,
                                     cmap=plt.cm.Blues,
                                     normalize=normalize)
        disp.ax_.set_title(title)
        print(title)
        print(disp.confusion_matrix)
    plt.show()
    return

### K Nearest Neighbors Classification

In [ ]:
# Import the class
from sklearn.neighbors import KNeighborsClassifier

# Instantiating the model (using the default parameters)
MODEL_KNN = KNeighborsClassifier(n_neighbors=5)

# Train the Model
MODEL_KNN.fit(X_train,y_train)

# Save the Trained Model
pickle.dump(MODEL_KNN, open('Model_KNeighborsClassifier.pkl', 'wb'))

# Predict the Trained Model on our Test data
y_pred_KNN = MODEL_KNN.predict(X_test)

# Print the Classification Report and Confusion Matrix
classifier_performance(MODEL_KNN,y_pred_KNN)

### Decision Trees Classification

In [ ]:
# Import the class
from sklearn.tree import DecisionTreeClassifier

# Instantiating the model (using the default parameters)
MODEL_DT = DecisionTreeClassifier()

# Train the Model
MODEL_DT.fit(X_train,y_train)

# Save the Trained Model
pickle.dump(MODEL_DT, open('DecisionTreeClassifier.pkl', 'wb'))

# Predict the Trained Model on our Test data
y_pred_DT = MODEL_DT.predict(X_test)

# Print the Classification Report and Confusion Matrix
classifier_performance(MODEL_DT,y_pred_DT)

### Gaussian Naive Bayes Classification

In [ ]:
# Import the class
from sklearn.naive_bayes import GaussianNB

# Instantiating the model (using the default parameters)
MODEL_GNB = GaussianNB()

# Train the Model
MODEL_GNB.fit(X_train,y_train)

# Save the Trained Model
pickle.dump(MODEL_GNB, open('Model_GaussianNB.pkl', 'wb'))

# Predict the Trained Model on our Test data
y_pred_GNB = MODEL_GNB.predict(X_test)

# Print the Classification Report and Confusion Matrix
classifier_performance(MODEL_GNB,y_pred_GNB)

### Support Vector Machines

In [ ]:
# Import the class
from sklearn.svm import SVC

# Instantiating the model (using the default parameters)
MODEL_SVM = SVC()

# Train the Model
MODEL_SVM.fit(X_train,y_train)

# Save the Trained Model
pickle.dump(MODEL_SVM, open('Model_SupportVectorClasifier.pkl', 'wb'))

# Predict the Trained Model on our Test data
y_pred_SVM = MODEL_SVM.predict(X_test)

# Print the Classification Report and Confusion Matrix
classifier_performance(MODEL_SVM,y_pred_SVM)